In [1]:
from dave.proxies.data import CrystalFeat
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from pathlib import Path
import torch
import numpy as np

/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BASE_PATH = Path(".").resolve().parent
DATA_PATH = BASE_PATH.parent / "ic_data" / "nrcc_ionic_conductivity"
print(DATA_PATH)

/home/minion/Downloads/ic_data/nrcc_ionic_conductivity


In [16]:
x_mean = torch.load(str(DATA_PATH / "x.mean"))
x_std = torch.load(str(DATA_PATH / "x.std"))

trainset = CrystalFeat(root=str(DATA_PATH), target="Ionic conductivity (S cm-1)", subset="train", scalex={"mean":x_mean, "std":x_std})
x_train = []
y_train = []

for i, (x, y) in enumerate(trainset):
    a, b, c = x
    # print(a[a != 0])
    # if i == 10:
    #     break
    b = b.unsqueeze(-1)
    x = torch.cat((a, b, c), dim=-1)
    x_train.append(x.numpy())
    y_train.append(y.numpy())
idx = np.arange(len(x_train))
print(max(idx) / 5)
np.random.shuffle(idx)
x_train = np.array(x_train)[idx]
y_train = np.array(y_train)[idx]

86.0


In [25]:
# hparams = {
#     "n_estimators": [50, 100, 150],
#     "max_depth": [12, 24, 36],
#     "max_features": ["log2", "sqrt"],
#     "min_samples_leaf": [2, 3, 4, 5]
# }

hparams = {
    'max_depth': 12,
    'max_features': 'sqrt',
    'min_samples_leaf': 2,
    'n_estimators': 50
}

model = RandomForestRegressor(**hparams)
res = cross_validate(model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error")
print(-np.mean((res["test_score"])))
# gs = GridSearchCV(estimator=model, param_grid=hparams, cv=5, scoring="")
# gs.fit(x_train, y_train)
# best = gs.best_score_
# print(best)
# print(gs.best_estimator_)
# print(gs.best_params_)
# y_train = np.array(y_train).reshape(-1, 1)
# y_pred = model.predict(x_train)


1.3860985837983129


In [33]:
hparams = {
    "hidden_layer_sizes": [128, 128, 128, 128, 128],
    "activation": "relu",
    "solver": "adam",
    "learning_rate_init": 0.003,
    # "validation_fraction" :0.2,
    "batch_size": 128,
    "max_iter": 40,
    "learning_rate": "adaptive"
}

model = MLPRegressor(**hparams)
res = cross_validate(model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error")
print(-np.mean((res["test_score"])))
# gs = GridSearchCV(estimator=model, param_grid=hparams, cv=5)
# model.fit(x_train[-86:], y_train[-86:])
# model.score(x_train[-86:], y_train[-86:])
# best = gs.best_score_
# print(best)
# print(gs.best_estimator_)
# print(gs.best_params_)

/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
 

1.7561132192611695


/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(


In [3]:
import numpy as np
from matminer.datasets import load_dataset
from modnet.models import MODNetModel
from modnet.preprocessing import MODData
from pymatgen.core import Composition

2024-05-28 21:20:14.802269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 21:20:15.459050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/openmpi/3.1.4/build/lib
2024-05-28 21:20:15.459151: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/openmpi/3.1.4/build/lib
2024-05-28 21:20:15.459157: W tensorflow/compiler/tf2tensorr

In [14]:
df = load_dataset("matbench_expt_gap")
df["composition"] = df["composition"].map(Composition) # maps composition to a pymatgen composition object
df = df.iloc[:10][:]
# print(df['composition'].map(lambda x: x.get_el_amt_dict()))

In [15]:
data = MODData(
    materials=df["composition"], # you can provide composition objects to MODData
    targets=df["gap expt"], 
    target_names=["gap_expt_eV"]
)

2024-05-28 21:25:32,825 - modnet - INFO - Loaded CompositionOnlyMatminer2023Featurizer featurizer.


/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/matminer/utils/data.py:281: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  mixing_dataset = pd.read_csv(


In [16]:
data.featurize()
print(data.df_featurized)

2024-05-28 21:25:42,381 - modnet - INFO - Computing features, this can take time...
2024-05-28 21:25:42,383 - modnet - INFO - Applying composition featurizers...
2024-05-28 21:25:42,386 - modnet - INFO - Applying featurizers (AtomicOrbitals(), AtomicPackingEfficiency(), BandCenter(), ElementFraction(), ElementProperty(data_source=<matminer.utils.data.MagpieData object at 0x7f1d72d38970>,
                features=['Number', 'MendeleevNumber', 'AtomicWeight',
                          'MeltingT', 'Column', 'Row', 'CovalentRadius',
                          'Electronegativity', 'NsValence', 'NpValence',
                          'NdValence', 'NfValence', 'NValence', 'NsUnfilled',
                          'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled',
                          'GSvolume_pa', 'GSbandgap', 'GSmagmom',
                          'SpaceGroupNumber'],
                stats=['minimum', 'maximum', 'range', 'mean', 'avg_dev',
                       'mode']), IonProperty(),

MultipleFeaturizer: 100%|██████████| 10/10 [00:00<00:00, 97.09it/s]


2024-05-28 21:25:45,711 - modnet - INFO - Data has successfully been featurized!
     AtomicOrbitals|HOMO_character  AtomicOrbitals|HOMO_element  \
id                                                                
id0                              2                           16   
id1                              3                           74   
id2                              2                           16   
id3                              2                           34   
id4                              1                           47   
id5                              3                           47   
id6                              2                           16   
id7                              2                           34   
id8                              2                           53   
id9                              3                           47   

     AtomicOrbitals|HOMO_energy  AtomicOrbitals|LUMO_character  \
id                                              